In [1]:
import pandas as pd
import gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import spacy
%run create_embeddings.ipynb
import preprocessing
import tfidf
import scipy
pd.set_option('max_colwidth', 5000)



In [2]:
#Importazione DB

#Data on Stack Overflow DB already preprocessed
data_stack = pd.read_csv('DB/Preprocessed_data.csv')

#data = data[data['processed_title'].notna()]

#Data on Ontology DB already preprocessed
data_onto = pd.read_csv('DB/Preprocessed_ontology.csv')

#Matrix for stack tfidf processing 
#tfidf_stack = scipy.sparse.load_npz('DB/tfidf_stack_matrix.npz')
stack_corpus = data_stack['processed_title'] + data_stack['questions_processed'] + data_stack['answers_processed'] 
stack_corpus = stack_corpus.dropna()

#Matrix for ontology tfidf processing
#tfidf_onto = scipy.sparse.load_npz('DB/tfidf_onto_matrix.npz')
onto_corpus = data_onto.description_processed

In [3]:
#Importazione WordEmbeddings created in file 'train_word2vec'

word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')
word_2_vec_model

In [4]:
#Fucntion to clear raw txt 
def normalize(txt):
    phrase = preprocessing.clear_text(txt)
    return phrase

In [5]:
#Fucntion to calculate cosine similarity between user raw txt and stack overflow processed title with word to vec  
def w2v_stack_result(txt):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = 5

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allTitleEmb = pd.read_csv('DB/titleEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli
    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allTitleEmb)[0])
    
    #Personalizzazione della misura di similarità
    similarityCosine = similarityCosine*(1+0.4*data_stack.score)
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    
    #Una volta calcolati gli score di similarità rendo i 10 più alti
    for i,similiarity_score in similarityCosine.nlargest(numberResult).iteritems():
        #print(i, data.Body[i])
        #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
        tags = data_stack.iloc[[i]].tags.item().split('|')
        #print(tags)
        for t in tags:
            if t in stringSearch:
                tags.remove(t)
                
        stack_id = data_stack.iloc[[i]].id.values[0]
        stack_title = data_stack.iloc[[i]].title.values[0]
        
        results.append(tuple([stack_id, similiarity_score, tags, stack_title ]))
        
    return results

In [6]:
def w2v_ontology_result(txt):
    stringSearch = normalize(txt)

    #print(stringSearch)

    numberResult = 3

    #Viene vettorizzata la query dell'utente

    vectorSearch = np.array([questionToVec(stringSearch, word_2_vec_model)])
    
    #Carico il file con gli embeddings dei titoli sui quali effettuare la corrispondenza
    allFuncEmb = pd.read_csv('DB/functionEmbeddings.csv').values

    #Calcolo della similarità del coseno for le query e tutti i titoli

    similarityCosine = pd.Series(cosine_similarity(vectorSearch, allFuncEmb)[0])
    
    #Lista che conterrà coppie i,j con i indice e j score di similarità
    results = []
    
      
    #Una volta calcolati gli score di similarità rendo i 10 più alti
    for i,similiarity_score in similarityCosine.nlargest(numberResult).iteritems():
        #print(i, data.Body[i])
        #Filtro i tag da rendere come risultato, eliminando quelli che sono già presenti nella string search per facilitare prossimi passaggi 
        description = data_onto.iloc[[i]].description.item()
        function = data_onto.iloc[[i]].function.item()
                    
        results.append(tuple([function,description, similiarity_score]))
      
    return results

In [7]:
def tfidf_stack_result(txt):
    stringSearch = normalize(txt)
    related_docs_indices = tfidf.get_results(stack_corpus, stringSearch)
    
    results = []
    
    for idx in related_docs_indices:
        
        stack_id = data_stack.iloc[[idx]].id.values[0]
        stack_title = data_stack.iloc[[idx]].title.values[0]
        tags = data_stack.iloc[[idx]].tags.item().split('|')
        
        results.append(tuple([stack_id, 0, tags, stack_title ]))
    
    return results

In [8]:
indices = tfidf_stack_result('how to delete nan values from series')
print(indices)

[ 9321 37621 21074 13685]


In [9]:
data_stack.loc[[13685]].title

13685    Putting lower bound and upper bounds on numpy.random.exponential
Name: title, dtype: object

In [18]:
w2v_stack_result('join column of two datasets')

[(45940141,
  0.9177789336288047,
  ['for-loop'],
  'How would I change this one line for loop to normal for loop?'),
 (45812581,
  0.9134695274094327,
  ['pandas', 'scikit-learn', 'feature-extraction'],
  'Replicated rows as dictionary in pandas for feature extraction'),
 (65957446,
  0.9024597896899457,
  ['string', 'for-loop', 'readlines'],
  'IndexError: list index out of range in a loop of readlines()'),
 (33485294,
  0.9014093577320642,
  ['django', 'heroku', 'gunicorn', 'mezzanine'],
  "Mezzanine - Can't load css and js in Heroku"),
 (45807268,
  0.8989503109464376,
  ['javascript', 'json'],
  'Issue with parsing (via JavaScript) an automatically created (via Python) JSON file')]

In [ ]:
#import joblib
#import scipy.sparse
#import tfidf

#matr = scipy.sparse.load_npz('DB/wc_matrix.npz')
#cvec = joblib.load("DB/count_vec.pkl")

#out = tfidf.tf_idf(matr,cvec,'how to delete nan values from series')
#print(out)